In [2]:
# ipython -c "%run plot.ipynb"
import pandas as pd

# Data Preprocessing
# with open("./avg-test-result-in-paper", "r") as f:
with open("./performance-comparison-table", "r") as f:
    df = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')

with open("../FIG-ST-MICRO/performance-comparison-table-fsync", "r") as f:
    df_fio = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')

with open("../FIG-Filebench/performance-comparison-table", "r") as f:
    df_filebench = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')

with open("../FIG-Trace/performance-comparison-table-fsync", "r") as f:
    df_trace = pd.read_csv(f, delim_whitespace=True, index_col=None, engine='python')

print(df_fio[(df_fio['file_system'] == "HUNTER-J") & (df_fio['ops'] == "seq-write")]['bandwidth(MiB/s)'])
print(df_fio[(df_fio['file_system'] == "HUNTER-SYNC") & (df_fio['ops'] == "seq-write")]['bandwidth(MiB/s)'])

def apply_df_fio(df, df_fio, fs):
    for i, fsize in enumerate([1024, 2048, 4096, 8192, 16384, 32768, 65536]):
        bw = df_fio[(df_fio['file_system'] == fs) & (df_fio['ops'] == "seq-write")]['bandwidth(MiB/s)'].iloc[i]
        file_system = fs
        workload = "fio-" + str(fsize)
        std_bw = df_fio[(df_fio['file_system'] == fs) & (df_fio['ops'] == "seq-write")]['std-bandwidth(MiB/s)'].iloc[i]
        new_row = pd.Series([file_system, workload, bw, std_bw], index=["file_system", "workloads", "perf(MiBPS/OPS)", "std-perf(MiBPS/OPS)"])
        df = df.append(new_row, ignore_index=True)
    return df

def apply_df_filebench(df, df_filebench, fs):
    for i, filebench in enumerate(["fileserver.f", "varmail.f", "webserver.f", "webproxy.f"]):
        ops = df_filebench[(df_filebench['file_system'] == fs) & (df_filebench['file_bench'] == filebench) & (df_filebench['threads'] == 1)]['iops'].iloc[0]
        file_system = "HUNTER-SYNC" if fs == "HUNTER" else fs
        workload = filebench
        std_ops = 0
        new_row = pd.Series([file_system, workload, ops, std_ops], index=["file_system", "workloads", "perf(MiBPS/OPS)", "std-perf(MiBPS/OPS)"])
        df = df.append(new_row, ignore_index=True)
    return df

def apply_df_trace(df, df_trace, fs):
    for i, trace in enumerate(["twitter", "facebook", "usr1", "usr2", "moodle", "gsf-filesrv"]):
        ops = df_trace[(df_trace['file_system'] == fs) & (df_trace['trace'] == trace)]['ops'].iloc[0]
        file_system = fs
        workload = trace + "-strict"
        std_ops = df_trace[(df_trace['file_system'] == fs) & (df_trace['trace'] == trace)]['std-ops'].iloc[0]
        new_row = pd.Series([file_system, workload, ops, std_ops], index=["file_system", "workloads", "perf(MiBPS/OPS)", "std-perf(MiBPS/OPS)"])
        df = df.append(new_row, ignore_index=True)
    return df

df = apply_df_fio(df, df_fio, "HUNTER-J")
df = apply_df_fio(df, df_fio, "HUNTER-SYNC")
df = apply_df_filebench(df, df_filebench, "HUNTER-J")
df = apply_df_filebench(df, df_filebench, "HUNTER")
df = apply_df_trace(df, df_trace, "HUNTER-J")
df = apply_df_trace(df, df_trace, "HUNTER-SYNC")
df.head()


28    1725.0
32    2068.0
36    1661.0
40    1635.0
44    1952.0
48    1968.0
52    1599.0
Name: bandwidth(MiB/s), dtype: float64
0     770.0
4     779.0
8     785.0
12    760.0
16    769.0
20    760.0
24    745.0
Name: bandwidth(MiB/s), dtype: float64


,file_system,workloads,perf(MiBPS/OPS),std-perf(MiBPS/OPS)
0,HUNTER-orig,fio-1024,1431.0,10.583005
1,HUNTER-orig,fio-2048,1448.0,46.669048
2,HUNTER-orig,fio-4096,1482.0,0.000000
3,HUNTER-orig,fio-8192,1417.0,67.882251
4,HUNTER-orig,fio-16384,1495.0,1.414214


In [8]:
for workload in [
    "varmail.f",
    "webserver.f",
    "webproxy.f",
    "twitter-strict",
    "facebook-strict",
    "usr1-strict",
    "usr2-strict",
    "moodle-strict",
    "gsf-filesrv-strict",
]:
    percentage_vheader = (df[(df['workloads'] == workload) & (df['file_system'] == "HUNTER-J")]['perf(MiBPS/OPS)'].iloc[0] - df[(df['workloads'] == workload) & (df['file_system'] == "HUNTER-vheader")]['perf(MiBPS/OPS)'].iloc[0]) / df[(df['workloads'] == workload) & (df['file_system'] == "HUNTER-vheader")]['perf(MiBPS/OPS)'].iloc[0]
    percentage_orig = (df[(df['workloads'] == workload) & (df['file_system'] == "HUNTER-J")]['perf(MiBPS/OPS)'].iloc[0] - df[(df['workloads'] == workload) & (df['file_system'] == "HUNTER-orig")]['perf(MiBPS/OPS)'].iloc[0]) / df[(df['workloads'] == workload) & (df['file_system'] == "HUNTER-orig")]['perf(MiBPS/OPS)'].iloc[0]
    if percentage_vheader > 0 or percentage_orig > 0:
        print(workload, percentage_vheader * 100, percentage_orig * 100)

webproxy.f -0.7616910654529597 1.4751316002836472
twitter-strict 4.874876695306832 6.809427688938599
facebook-strict 1.2972075464336874 2.5854157752967826
usr1-strict 0.7281240976438882 3.486564862681825
usr2-strict -0.25664505549925765 2.837291474034759
moodle-strict 3.8812721595487187 3.58269643588763
gsf-filesrv-strict 0.899508809335554 1.6286559643420042
